## Examples for ```mat2qubit```

```mat2qubit``` is a Python package for converting a set of matrix operators into a Pauli operator representation. Intended for use in physics, chemistry, materials, linear algebra problems, as well as classical optimization problems over discrete (d>2) variables. The current version assumes that operators on different subsystems always commute (hence the code cannot be used for fermionic problems).

The code performs mappings of the following class:

$$\sum_i b_i \cdots \otimes M \otimes N \otimes \cdots \rightarrow \sum_j c_j \bigotimes \{I,\sigma_x,\sigma_y,\sigma_z\}$$

where $M,N,\cdots$ are square matrices of arbitrary size.

The Pauli matrices:
$$
\sigma_x \equiv X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}
;
\sigma_y \equiv Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}
;
\sigma_z \equiv Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}
;
I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}
$$

`mat2qubit` can be used to implement different encodings for which there are often depth-space tradeoffs or other resource tradeoffs (see references below). The built-in encodings include <b>standard binary</b>, <b>Gray code</b>, <b>unary (one-hot)</b>, and <b>block unary</b>. User-defined embeddings may be added by modifying `integer2bit.py`.

Mathematical and theoretical details can be found in the following references. If you use the code, please cite an appropriate subset of these.
1. Sawaya, NPD. <i>mat2qubit: A lightweight pythonic package for qubit encodings of vibrational, bosonic, graph coloring, routing, scheduling, and general matrix problems.</i> [arXiv:2205.09776](https://arxiv.org/abs/2205.09776) (2022).
2. Sawaya, NPD, Menke, T, Kyaw, TH, Johri, S, Aspuru-Guzik, A and Guerreschi, GG. [npj Quantum Information, 6(49)](https://www.nature.com/articles/s41534-020-0278-0) (2020).
3. Sawaya, NPD, Schmitz, AT, and Hadfield, S. arXiv preprint [arXiv:2203.14432](https://arxiv.org/abs/2203.14432) (2022).

In [1]:
# Copyright (C) 2020-2022 Intel Corporation
# SPDX-License-Identifier: Apache-2.0

import mat2qubit as m2q
from openfermion import QubitOperator
import numpy as np
import itertools

# The primary classes of mat2qubit:
# * dLevelSubsystem - Single particle object
# * compositeDLevels - Collection of quantum "particles" (does not store operator terms)
# * compositeOperator - Multi-particle operator (child class of compositeDLevels)

### 1. Built-in functions.

$a^\dagger = \begin{pmatrix} 0 & 0 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & \sqrt{2} & 0 & 0 \\ 0 & 0 & \sqrt{3} & 0 \end{pmatrix}$
$\rightarrow$ Standard binary

In [2]:
# Declare 4-level subsystem, standard binary encoding
four_level_particle = m2q.dLevelSubsystem(d=4,enc="stdbinary")
print("Standard binary (SB) encoding, QHO Creation operator (not Hermitian)\n")
print(four_level_particle.opToPauli("ad"))  # "ad" for 'a^\dagger' (Alternative: "qhoCr")

Standard binary (SB) encoding, QHO Creation operator (not Hermitian)

0.6830127018922193 [X0] +
0.3535533905932738 [X0 X1] +
-0.3535533905932738j [X0 Y1] +
-0.1830127018922193 [X0 Z1] +
-0.6830127018922193j [Y0] +
0.3535533905932738j [Y0 X1] +
(0.3535533905932738+0j) [Y0 Y1] +
0.1830127018922193j [Y0 Z1]


### 2. User-defined matrix (general linear algebra)

Random 4-by-4 matrix $\rightarrow$ Gray code

In [3]:
# Gray encoding of random Hermitian operator
rand_matrix = np.random.rand(4,4)  # Random marix
rand_matrix = rand_matrix + rand_matrix.T  # Hermitian
four_level_particle.setEncoding("gray") # Change encoding
print("Gray encoding, random Hermitian operator\n")
print(four_level_particle.opToPauli(rand_matrix))

Gray encoding, random Hermitian operator

1.0078228712238775 [] +
1.5969794105029569 [X0] +
1.2487470669113498 [X0 X1] +
0.20070869159609356 [X0 Z1] +
(0.16191438502379096+0j) [Y0 Y1] +
0.07909151803400977 [Z0] +
0.12200608341171215 [Z0 X1] +
0.17552850267072334 [Z0 Z1] +
0.9957407101977428 [X1] +
0.4532308269799442 [Z1]


### 3. The good stuff: Building composite systems.

$k(a^\dagger_0 a_1 + a^\dagger_1 a_0)$ $\rightarrow$ Unary

In [4]:
# ** Two-site, two-level systems, a_0^ a_1, unary
# Create a 2-level particle
ss1 = m2q.dLevelSubsystem(2, "unary")

# Create composite system of two particles
twoSite = m2q.compositeDLevels()
twoSite.appendSubsystem(ss1) # Site 1, deepcopied
twoSite.appendSubsystem(ss1) # Site 2, deepcopied

# Get a Pauli operator from a single term (list of tuples)
opString = [(0,"qhoCr"),(1,"qhoAn")]
res = twoSite.opStringToPauli(2.0,opString)
print("2 [a_0^ a_1] --> unary \n")
print(res,"\n")

# Now create a composite operator of multiple particles
# print(type(twoSite))
# print(super(compositeOperator))
twoSiteOperator = m2q.compositeOperator(twoSite)
k = 4.2
twoSiteOperator.addHamTerm(k, [(1,"qhoCr"),(0,"qhoAn")] )
twoSiteOperator.addHamTerm(k, [(0,"qhoCr"),(1,"qhoAn")] )
twoSiteOperator.addHamTerm(0.5, "ident" ) # Identity
print("2 [a_0^ a_1 + a_1^ a_0] + 0.5 [] --> unary \n")
print(twoSiteOperator.opToPauli())


2 [a_0^ a_1] --> unary 

0.125 [X0 X1 X2 X3] +
0.125j [X0 X1 X2 Y3] +
-0.125j [X0 X1 Y2 X3] +
(0.125+0j) [X0 X1 Y2 Y3] +
-0.125j [X0 Y1 X2 X3] +
(0.125+0j) [X0 Y1 X2 Y3] +
(-0.125-0j) [X0 Y1 Y2 X3] +
-0.125j [X0 Y1 Y2 Y3] +
0.125j [Y0 X1 X2 X3] +
(-0.125+0j) [Y0 X1 X2 Y3] +
(0.125+0j) [Y0 X1 Y2 X3] +
0.125j [Y0 X1 Y2 Y3] +
(0.125+0j) [Y0 Y1 X2 X3] +
0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3] +
(0.125+0j) [Y0 Y1 Y2 Y3] 

2 [a_0^ a_1 + a_1^ a_0] + 0.5 [] --> unary 

0.5 [] +
0.525 [X0 X1 X2 X3] +
(0.525+0j) [X0 X1 Y2 Y3] +
(0.525+0j) [X0 Y1 X2 Y3] +
(-0.525+0j) [X0 Y1 Y2 X3] +
(-0.525+0j) [Y0 X1 X2 Y3] +
(0.525+0j) [Y0 X1 Y2 X3] +
(0.525+0j) [Y0 Y1 X2 X3] +
(0.525+0j) [Y0 Y1 Y2 Y3]


### 4. Bose-Hubbard model

This example makes use the of qSymbOp class (see qSymbOp_examples.ipynb). The progression is:

{Fully symbolic repr} $\rightarrow$ {symb w/ numerical coefficients} $\rightarrow$ {compositeOperator} $\rightarrow$ {QubitOperator}

3 particles, with periodic boundary conditions.
$$
H = t \sum_{i \neq j} b_i^\dagger b_j + \frac{U}{2} \sum_i n_i^2
$$

In [5]:
# Problem parameters
t = 1
U = 0.5
M = 3   # 3 sites
d = 4   # 4 levels per boson

# Initialize a symbolic operator
bh_model = m2q.qSymbOp("")
for i in range(M):
    j = (i+1)%M
    bh_model += m2q.qSymbOp(f" t [a_{i} ad_{j}] ++ t [ad_{i} a_{j}] ++ U/2 [Nsq_{i}]")

print('''
The symbolic operator for a 3-particle BH model
(defined before any choice of d [# levels] or encoding)''')
print(bh_model)

print("\nSubstitute numerical values for coefficients (in-place function)")
vals = {'t':t, 'U':U}
bh_model.scalar_subs(vals) 
print(bh_model)

print("\nConvert to mat2qubit compositeOperator")
ssid_order = [str(i) for i in range(3)] # ['0','1','2']
dlev_obj = m2q.symbop_to_dlevcompositeop(bh_model, ssid_order,4,'gray')
print(dlev_obj)

print("\nConvert to QubitOperator")
qub_op = dlev_obj.opToPauli()
print(qub_op)


The symbolic operator for a 3-particle BH model
(defined before any choice of d [# levels] or encoding)
(t) [a_0 ad_1]
++ (t) [ad_0 a_1]
++ (U/2) [Nsq_0]
++ (t) [a_1 ad_2]
++ (t) [ad_1 a_2]
++ (U/2) [Nsq_1]
++ (t) [a_2 ad_0]
++ (t) [ad_2 a_0]
++ (U/2) [Nsq_2]

Substitute numerical values for coefficients (in-place function)
((1+0j)) [a_0 ad_1]
++ ((1+0j)) [ad_0 a_1]
++ ((0.25+0j)) [Nsq_0]
++ ((1+0j)) [a_1 ad_2]
++ ((1+0j)) [ad_1 a_2]
++ ((0.25+0j)) [Nsq_1]
++ ((1+0j)) [a_2 ad_0]
++ ((1+0j)) [ad_2 a_0]
++ ((0.25+0j)) [Nsq_2]

Convert to mat2qubit compositeOperator
[(1.0, ((0, 'qhoAn'), (1, 'qhoCr'))), (1.0, ((0, 'qhoCr'), (1, 'qhoAn'))), (0.25, ((0, 'Nsq'),)), (1.0, ((1, 'qhoAn'), (2, 'qhoCr'))), (1.0, ((1, 'qhoCr'), (2, 'qhoAn'))), (0.25, ((1, 'Nsq'),)), (1.0, ((2, 'qhoAn'), (0, 'qhoCr'))), (1.0, ((2, 'qhoCr'), (0, 'qhoAn'))), (0.25, ((2, 'Nsq'),))]

Convert to QubitOperator
2.625 [] +
-0.24999999999999994 [X0 Z1 X2] +
0.06698729810778066 [X0 Z1 X2 Z3] +
0.12940952255126037 [X0 Z1 Z2 

### 5. Anharmonic molecular vibrations
This example implements the potential energy surface for sulfur dioxide (SO$_2$). There are there vibrational modes. In the harmonic basis, the general anharmonic vibrational Hamiltonian is
$$
H = \frac{1}{2}\sum_k^M(q_{k}^2 + p_k^2) + \sum_{ijk} k_{ijk}q_iq_jq_k + \dots
$$
where $q$ and $p$ are position and momentum operators, respectively.

References:
* Sawaya and Huh, J. Phys. Chem. Lett. 2019, 10, 13, 3586–3591.
* Smith, Liu, and Noid. Chemical Physics, 1984, 89(3), 345– 351.

In [6]:
# Harmonic:
freqA = 1171
freqB = 525
freqC = 1378

harmonic_terms = m2q.qSymbOp(
f'''({freqA}) [q2_a] ++ ({freqA}) [p2_a] 
++  ({freqB}) [q2_b] ++ ({freqB}) [p2_b] 
++  ({freqC}) [q2_c] ++ ({freqC}) [p2_c]'''
)
    
# anharmonic:
# k111 k112 k122 k133 k222 k233 k1111 k1122 k1133 k2222 k2233 k3333
# 44   -19  -12  159  -7.0 4.7  1.8   -3.1  15    -1.4  -6.5   3.0

anharmonic_terms = m2q.qSymbOp(
'''44 [q3_a] 
++ -19 [q2_a q_b] 
++ -12 [q_a q2_b] 
++ 159 [q_a q2_c] 
++ -7.0 [q3_b] 
++ 4.7 [q_b q2_c] 
++ 1.8 [q4_a] 
++ -3.1 [q2_a q2_b] 
++ 15 [q2_a q2_c] 
++ -1.4 [q4_b] 
++ -6.5 [q2_b q2_c] 
++ 3.0 [q4_c]'''
)

print("\nSymbolic Hamiltonian")
symb_hamiltonian = harmonic_terms + anharmonic_terms
print(symb_hamiltonian)
# (do all up to 16)

print("\nQubit Operator [Gray code; d=(8,16,8)]")
ssid_order = ('a','b','c')
d_values = (4,8,4)
dlev_obj = m2q.symbop_to_dlevcompositeop(symb_hamiltonian, ssid_order,d_values,'gray')
qub_op = dlev_obj.opToPauli()
print(qub_op)



Symbolic Hamiltonian
((1171+0j)) [q2_a]
++ ((1171+0j)) [p2_a]
++ ((525+0j)) [q2_b]
++ ((525+0j)) [p2_b]
++ ((1378+0j)) [q2_c]
++ ((1378+0j)) [p2_c]
++ ((44+0j)) [q3_a]
++ ((-19+0j)) [q2_a q_b]
++ ((-12+0j)) [q_a q2_b]
++ ((159+0j)) [q_a q2_c]
++ ((-7+0j)) [q3_b]
++ ((4.7+0j)) [q_b q2_c]
++ ((1.8+0j)) [q4_a]
++ ((-3.1+0j)) [q2_a q2_b]
++ ((15+0j)) [q2_a q2_c]
++ ((-1.4+0j)) [q4_b]
++ ((-6.5+0j)) [q2_b q2_c]
++ ((3+0j)) [q4_c]

Qubit Operator [Gray code; d=(8,16,8)]
(14373.650000000001+0j) [] +
405.38423914497173 [X0] +
24.42083477315343 [X0 X1] +
-24.701757434527643 [X0 X1 X2] +
-5.9219917136363645 [X0 X1 X2 X3] +
3.0296523377704854 [X0 X1 X2 X3 Z4] +
1.0458601272530461 [X0 X1 X2 Z3] +
2.970504022617485 [X0 X1 X2 Z3 X4] +
3.704139872746954 [X0 X1 X2 Z3 Z4] +
-2.970504022617485 [X0 X1 X2 X4] +
6.974516098575485 [X0 X1 X2 Z4] +
(-0.01188804479545243+0j) [X0 X1 Y2 Y3] +
(-0.7631119552045474+0j) [X0 X1 Y2 Y3 Z4] +
(-0.37730348103028055+0j) [X0 X1 Y2 Z3 Y4] +
(0.37730348103028055+0j) [X0 X1

### 6. Spin-3/2 and spin-5/2 models

Transverse-field spin model with spin-3/2 and spin-5/2 Hamiltonians. The model has two spins $a$ and $b$, with the same values for $s$.

$$
H = \hat S_z^{a} \hat S_z^{b} + g \hat S_x^{a} + g \hat S_x^{b}
$$

Note: In `mat2qubit` the spin $s$ of the particle is determined by the user's definition of the number of levels $d$:
$$
s = \{1/2, 1, 3/2, 2, 5/2, 3, \dots \}  \leftrightarrow d = \{2,3,4,5,6,7,\dots\}
$$

In [7]:
# Hamiltonian
spin_symb = m2q.qSymbOp(" [Sz_a Sz_b] ++ g [Sx_a] ++ g [Sx_b]")
spin_symb.scalar_subs({'g': 3.0})

print("\nspin-5/2 (d=4) Hamiltonian with standard binary encoding")
d = 4 # Corresponds to s=3/2
dlev_obj = m2q.symbop_to_dlevcompositeop(spin_symb, ('a','b'),d,'stdbinary')
qub_op = dlev_obj.opToPauli()
print(qub_op)

print("\nspin-5/2 (d=6) Hamiltonian with unary encoding")
d = 6 # Corresponds to s=5/2
dlev_obj = m2q.symbop_to_dlevcompositeop(spin_symb, ('a','b'),d,'unary')
qub_op = dlev_obj.opToPauli()
print(qub_op)




spin-5/2 (d=4) Hamiltonian with standard binary encoding
(2.598076211353316+0j) [X0] +
(1.5+0j) [X0 X1] +
(1.5+0j) [Y0 Y1] +
(0.25+0j) [Z0 Z2] +
(0.5+0j) [Z0 Z3] +
(0.5+0j) [Z1 Z2] +
(1+0j) [Z1 Z3] +
(2.598076211353316+0j) [X2] +
(1.5+0j) [X2 X3] +
(1.5+0j) [Y2 Y3]

spin-5/2 (d=6) Hamiltonian with unary encoding
(1.6770509831248424+0j) [X0 X1] +
(1.6770509831248424+0j) [Y0 Y1] +
(1.5625+0j) [Z0 Z6] +
(0.9375+0j) [Z0 Z7] +
(0.3125+0j) [Z0 Z8] +
(-0.3125+0j) [Z0 Z9] +
(-0.9375+0j) [Z0 Z10] +
(-1.5625+0j) [Z0 Z11] +
(2.121320343559643+0j) [X1 X2] +
(2.121320343559643+0j) [Y1 Y2] +
(0.9375+0j) [Z1 Z6] +
(0.5625+0j) [Z1 Z7] +
(0.1875+0j) [Z1 Z8] +
(-0.1875+0j) [Z1 Z9] +
(-0.5625+0j) [Z1 Z10] +
(-0.9375+0j) [Z1 Z11] +
(2.25+0j) [X2 X3] +
(2.25+0j) [Y2 Y3] +
(0.3125+0j) [Z2 Z6] +
(0.1875+0j) [Z2 Z7] +
(0.0625+0j) [Z2 Z8] +
(-0.0625+0j) [Z2 Z9] +
(-0.1875+0j) [Z2 Z10] +
(-0.3125+0j) [Z2 Z11] +
(2.121320343559643+0j) [X3 X4] +
(2.121320343559643+0j) [Y3 Y4] +
(-0.3125+0j) [Z3 Z6] +
(-0.1875+0j

### 7. Sum of tensor trains (general classical linear algebra) 


Any classical square matrix $A$ of dimension $N \times N$ may be decomposed as

$$
M \mapsto \sum_j c_j \hat a_{j,1} \otimes \hat a_{j,2} \otimes \cdots
$$
where $\hat a_i$ are square matrices and $\prod_i \textrm{dim}(\hat a_{j,i})=N$. For instance, a $64 \times 64$ matrix may be decomposed this way, using three subspaces each of dimension 4 (since $4^3=64$). If the decomposition is of low rank, then it may be efficient to implement on a quantum computer.

The example below is a sum over 3 tensor trains, over 4 "subspaces", where each tensor in each train is a random $2 \times 2$ matrix, superscripts denote the member in the sum, and subscripts denote the $d=4$ subystem id $\in \{1,2,3,4\}$.

$$
M = \hat a^{(1)}_1 \otimes \hat b^{(1)}_2 \otimes I \otimes I + I \otimes \hat a^{(2)}_2 \otimes \hat b^{(2)}_3 \otimes I + I \otimes I \otimes \hat a^{(3)}_3 \otimes \hat b^{(3)}_4
$$


We introduce this example partly to highlight the use of `mat2qubit` for classical linear algebra, and partly to demonstrate the use of the `inpOpChars` argument in `symbop_to_dlevcompositeop()`.

In [8]:

# Define operator
d = 2
ttrain_symb = m2q.qSymbOp("[a1_1 b1_2] ++ [a2_2 b2_3] ++ [a3_3 b3_4]")

# Specify random matrices of the tensor trains
inpOps = {}
inpOps['a1'] = np.random.rand(d,d)
inpOps['b1'] = np.random.rand(d,d)
inpOps['a2'] = np.random.rand(d,d)
inpOps['b2'] = np.random.rand(d,d)
inpOps['a3'] = np.random.rand(d,d)
inpOps['b3'] = np.random.rand(d,d)

# ttrain_symb
m2q_op = m2q.symbop_to_dlevcompositeop(ttrain_symb,
                                       [str(i) for i in range(1,5)],d,'stdbinary',
                                       inpOpChars=inpOps )
qub_op = m2q_op.opToPauli()
print("\nQubit repr of sum of random local tensor trains (not Hermitian)")
print(qub_op)



Qubit repr of sum of random local tensor trains (not Hermitian)
0.5872107059945466 [] +
0.5161344530631551 [X0] +
0.306550973983633 [X0 X1] +
0.17218955389559618j [X0 Y1] +
0.09273179667388837 [X0 Z1] +
-0.12241123856800563j [Y0] +
-0.07270447494225565j [Y0 X1] +
(0.040838073172094855+0j) [Y0 Y1] +
-0.02199313380092873j [Y0 Z1] +
0.18615944001041226 [Z0] +
0.11056684418712252 [Z0 X1] +
0.0621053501439568j [Z0 Y1] +
0.03344651618879267 [Z0 Z1] +
0.5103274997153506 [X1] +
0.35258359756816987 [X1 X2] +
0.1519615372444082j [X1 Y2] +
0.26937576280412856 [X1 Z2] +
0.004153505724685119j [Y1] +
-0.08736951762723082j [Y1 X2] +
(0.03765576815968992+0j) [Y1 Y2] +
-0.06675078086159034j [Y1 Z2] +
-0.015164846345900623 [Z1] +
-0.06463386712679463 [Z1 X2] +
-0.02785683132846115j [Z1 Y2] +
-0.049380621731544734 [Z1 Z2] +
0.28289679838138415 [X2] +
0.029522716337189555 [X2 X3] +
0.020509216764734838j [X2 Y3] +
0.006341929264918945 [X2 Z3] +
0.11815879992728981j [Y2] +
0.01059271448688128j [Y2 X3] +
(-

### 8. Graph coloring: cost function
This example is of a A 4-node graph known as the paw graph \[a.k.a. 3-pan graph or (3,1)-lollipop graph or (3,1)-tadppole graph\]. It is <i>3-colorable</i>; hence we use 3 colors ($d$=3) in our cost function.

When implementing QAOA, we recommend using graph-derived partial mixers (GDPMs)
of arXiv:2203.14432 when $d$ is not a power of 2.

The `Pr{i}_{j}` operator means the projector of value $i$ on subspace $j$. For example `Pr3_2` refers to $| 3 \rangle \langle 3 |_2$.

In [9]:
def EQ(d,n1,n2, **kwargs):
    eq_str = ""
    for i in range(d):
        eq_str += " ++ [Pr{0}_{1} Pr{0}_{2}]".format(i,n1,n2)
    return m2q.qSymbOp(eq_str)

def coloring_cost(adjmat,ncolor):
    assert adjmat.shape[0]==adjmat.shape[1]
    N = adjmat.shape[0]

    cost_ham = m2q.qSymbOp()

    for i,j in itertools.product(range(N),repeat=2):
        w = adjmat[i,j]
        if j>=i: # Upper-triangular is ignored
            continue
        if w==0:
            continue
        assert w==1
        cost_ham += EQ(ncolor, j,i)

    return cost_ham

# The paw graph
adjacency_graph = np.array([[0,1,1,0],
                            [1,0,1,0],
                            [1,1,0,1],
                            [0,0,1,0]])
# 4 nodes, 3 colors
k  = 3
nn = 4

# Get cost function
print('''
Cost function for 3 colors. Each node has 3 possible levels (colors) 
and there are 4 nodes (_0,_1,_2,_3) in the graph.
''')
cost_3pan_3color = coloring_cost(adjacency_graph,3)
print(cost_3pan_3color)

# Convert to qubit operator
print("\n***** Standard binary *****")
m2q_op = m2q.symbop_to_dlevcompositeop(cost_3pan_3color,
                                       [str(i) for i in range(nn)],k,'stdbinary')

qub_op = m2q_op.opToPauli()
print(qub_op)
print("\n***** Gray *****")
m2q_op.setEncodingForAllSS('gray')
qub_op = m2q_op.opToPauli()
print(qub_op)
print("\n***** Unary (one-hot) *****")
m2q_op.setEncodingForAllSS('unary')
qub_op = m2q_op.opToPauli()
print(qub_op)


Cost function for 3 colors. Each node has 3 possible levels (colors) 
and there are 4 nodes (_0,_1,_2,_3) in the graph.

((1+0j)) [Pr0_0 Pr0_1]
++ ((1+0j)) [Pr1_0 Pr1_1]
++ ((1+0j)) [Pr2_0 Pr2_1]
++ ((1+0j)) [Pr0_0 Pr0_2]
++ ((1+0j)) [Pr1_0 Pr1_2]
++ ((1+0j)) [Pr2_0 Pr2_2]
++ ((1+0j)) [Pr0_1 Pr0_2]
++ ((1+0j)) [Pr1_1 Pr1_2]
++ ((1+0j)) [Pr2_1 Pr2_2]
++ ((1+0j)) [Pr0_2 Pr0_3]
++ ((1+0j)) [Pr1_2 Pr1_3]
++ ((1+0j)) [Pr2_2 Pr2_3]

***** Standard binary *****
(0.75+0j) [] +
(0.125+0j) [Z0] +
(-0.125+0j) [Z0 Z1] +
(0.0625+0j) [Z0 Z1 Z2] +
(0.1875+0j) [Z0 Z1 Z2 Z3] +
(0.0625+0j) [Z0 Z1 Z3] +
(0.0625+0j) [Z0 Z1 Z4] +
(0.1875+0j) [Z0 Z1 Z4 Z5] +
(0.0625+0j) [Z0 Z1 Z5] +
(0.1875+0j) [Z0 Z2] +
(0.0625+0j) [Z0 Z2 Z3] +
(-0.0625+0j) [Z0 Z3] +
(0.1875+0j) [Z0 Z4] +
(0.0625+0j) [Z0 Z4 Z5] +
(-0.0625+0j) [Z0 Z5] +
(0.125+0j) [Z1] +
(-0.0625+0j) [Z1 Z2] +
(0.0625+0j) [Z1 Z2 Z3] +
(0.1875+0j) [Z1 Z3] +
(-0.0625+0j) [Z1 Z4] +
(0.0625+0j) [Z1 Z4 Z5] +
(0.1875+0j) [Z1 Z5] +
(0.125+0j) [Z2] +
(-0.125+0j) [

### 9. Traveling salesperson problem (traveling salesman): cost function

When implementing QAOA, we suggest using graph-derived partial mixers (GDPMs)
of arXiv:2203.14432.


In [10]:
def PERMUTATION_PENALTY(N,pen_val=1.0, **kwargs):
    '''Pair permutation penalty
    '''
    pen = m2q.qSymbOp()
    for i,j,u in itertools.product(range(N),repeat=3):
        if j>=i:
            continue # Each (i,j) position pair counted only once
        pen += m2q.qSymbOp( "({}) [Pr{}_{} Pr{}_{}]".format(pen_val, u,j, u,i) )
    return pen

def trav_sales_cost(dist_mat,penal_perm=0.):
    '''Returns cost function for TSP (traverling salesperson problem)
    '''

    assert dist_mat.shape[0]==dist_mat.shape[1], "dist_mat must be square."

    N = dist_mat.shape[0]

    tsp_cost = m2q.qSymbOp()

    for i,u,v in itertools.product(range(N),repeat=3):
        if v>=u: # Upper-triangular is ignored
            continue
        #tsp_cost += heis.qSymbOp( "({}) [Pr{}_{} Pr{}_{}]".format(dist_mat[u,v], u,i, v, (i+1)%N ) )
        # Note this order of v&u makes it easier for the unit tests:
        tsp_cost += m2q.qSymbOp( "({}) [Pr{}_{} Pr{}_{}]".format(dist_mat[u,v], v,i, u, (i+1)%N ) )

    # Include penalty if !=0
    if penal_perm!=0:
        tsp_cost += PERMUTATION_PENALTY(N,pen_val=penal_perm)

    return tsp_cost

dist_mat = np.array([[0,0,0],
                     [3,0,0],
                     [4,5,0]],dtype=complex)

# *****
# Without penalty
cost_op = trav_sales_cost(dist_mat,0.)

# Order terms
cost_op.orderTerms()

#
print("\nqSymbOp without penalty")
print(cost_op)

# *****
# With penalty
pen = 10
cost_op = trav_sales_cost(dist_mat,pen)

# Order terms
cost_op.orderTerms()

#
print("\nqSymbOp with pairwise permutation penalty")
print(cost_op)

# User may then use m2q.symbop_to_dlevcompositeop().opToPauli() to obtain qubit operator.



qSymbOp without penalty
((3+0j)) [Pr0_0 Pr1_1]
++ ((4+0j)) [Pr0_0 Pr2_1]
++ ((5+0j)) [Pr1_0 Pr2_1]
++ ((3+0j)) [Pr0_1 Pr1_2]
++ ((4+0j)) [Pr0_1 Pr2_2]
++ ((5+0j)) [Pr1_1 Pr2_2]
++ ((3+0j)) [Pr0_2 Pr1_0]
++ ((4+0j)) [Pr0_2 Pr2_0]
++ ((5+0j)) [Pr1_2 Pr2_0]

qSymbOp with pairwise permutation penalty
((10+0j)) [Pr0_0 Pr0_1]
++ ((3+0j)) [Pr0_0 Pr1_1]
++ ((4+0j)) [Pr0_0 Pr2_1]
++ ((10+0j)) [Pr0_0 Pr0_2]
++ ((10+0j)) [Pr1_0 Pr1_1]
++ ((5+0j)) [Pr1_0 Pr2_1]
++ ((10+0j)) [Pr1_0 Pr1_2]
++ ((10+0j)) [Pr2_0 Pr2_1]
++ ((10+0j)) [Pr2_0 Pr2_2]
++ ((10+0j)) [Pr0_1 Pr0_2]
++ ((3+0j)) [Pr0_1 Pr1_2]
++ ((4+0j)) [Pr0_1 Pr2_2]
++ ((10+0j)) [Pr1_1 Pr1_2]
++ ((5+0j)) [Pr1_1 Pr2_2]
++ ((10+0j)) [Pr2_1 Pr2_2]
++ ((3+0j)) [Pr0_2 Pr1_0]
++ ((4+0j)) [Pr0_2 Pr2_0]
++ ((5+0j)) [Pr1_2 Pr2_0]


### 10. Single machine scheduling: cost function

When implementing QAOA, we suggest using graph-derived partial mixers (GDPMs)
of arXiv:2203.14432.


In [11]:
def factory_scheduling_cost(proc_times, deadlines, weights=None, penal_perm=0., **kwargs):
    '''This function minimizes lateness (as opposed to tardiness)
    '''

    assert len(proc_times)==len(deadlines), (len(proc_times),len(deadlines))

    if weights:
        raise Exception("Weights not yet implemented.")
        assert len(proc_times)==len(deadlines)==len(weights), (len(proc_times),len(deadlines),len(weights))
        weights = np.ones(len(proc_times))

    # Number of jobs
    N = len(proc_times)

    sched_cost = m2q.qSymbOp()

    for k,alpha in itertools.product(range(N),repeat=2):

        # Start times
        for beta in range(alpha):  # beta < alpha
            for k_prime in range(N):
                sched_cost += m2q.qSymbOp( "({}) [Pr{}_{} Pr{}_{}] ".format(proc_times[k],k,alpha,k_prime,beta ) )

    for k,alpha in itertools.product(range(N),repeat=2):
        # Processing time
        sched_cost += m2q.qSymbOp("({}) [Pr{}_{}]".format( proc_times[k], k,alpha ) )

        # Deadline
        sched_cost += (-1)*m2q.qSymbOp("({}) [Pr{}_{}]".format( deadlines[k], k,alpha ) )


    # Penalty (for invalid permutation)
    if penal_perm!=0:
        for i,j,u in itertools.product(range(N),repeat=3):
            if j>=i:
                continue # Each (i,j) position pair counted only once
            tsp_cost += m2q.qSymbOp( "({}) [Pr{}_{} Pr{}_{}]".format(penal_perm, u,j, u,i) )

    return sched_cost


proc_times = ['p0','p1','p2']
deadlines  = ['d0','d1','d2']
fac_cost = factory_scheduling_cost(proc_times,deadlines)

print("\nScheduling cost function with all symbolic coefficients (3 jobs)")
print( str(fac_cost) )

# User may then use scalar_subs(), symbop_to_dlevcompositeop(), and opToPauli() to obtain qubit operator.



Scheduling cost function with all symbolic coefficients (3 jobs)
(p0) [Pr0_1 Pr0_0]
++ (p0) [Pr0_1 Pr1_0]
++ (p0) [Pr0_1 Pr2_0]
++ (p0) [Pr0_2 Pr0_0]
++ (p0) [Pr0_2 Pr1_0]
++ (p0) [Pr0_2 Pr2_0]
++ (p0) [Pr0_2 Pr0_1]
++ (p0) [Pr0_2 Pr1_1]
++ (p0) [Pr0_2 Pr2_1]
++ (p1) [Pr1_1 Pr0_0]
++ (p1) [Pr1_1 Pr1_0]
++ (p1) [Pr1_1 Pr2_0]
++ (p1) [Pr1_2 Pr0_0]
++ (p1) [Pr1_2 Pr1_0]
++ (p1) [Pr1_2 Pr2_0]
++ (p1) [Pr1_2 Pr0_1]
++ (p1) [Pr1_2 Pr1_1]
++ (p1) [Pr1_2 Pr2_1]
++ (p2) [Pr2_1 Pr0_0]
++ (p2) [Pr2_1 Pr1_0]
++ (p2) [Pr2_1 Pr2_0]
++ (p2) [Pr2_2 Pr0_0]
++ (p2) [Pr2_2 Pr1_0]
++ (p2) [Pr2_2 Pr2_0]
++ (p2) [Pr2_2 Pr0_1]
++ (p2) [Pr2_2 Pr1_1]
++ (p2) [Pr2_2 Pr2_1]
++ (-d0 + p0) [Pr0_0]
++ (-d0 + p0) [Pr0_1]
++ (-d0 + p0) [Pr0_2]
++ (-d1 + p1) [Pr1_0]
++ (-d1 + p1) [Pr1_1]
++ (-d1 + p1) [Pr1_2]
++ (-d2 + p2) [Pr2_0]
++ (-d2 + p2) [Pr2_1]
++ (-d2 + p2) [Pr2_2]
